# Chapter 3 Exercises

Import of book's data structures to be used. Renamed the new list type to NewList becuse scala is weird.

In [25]:
sealed trait NewList[+A] // `NewList` data type, parameterized on a type, `A`
case object Nil extends NewList[Nothing] // A `NewList` data constructor representing the empty list
/* Another data constructor, representing nonempty lists. Note that `tail` is another `NewList[A]`,
which may be `Nil` or another `Cons`.
*/
case class Cons[+A](head: A, tail: NewList[A]) extends NewList[A]

object NewList { // `NewList` companion object. Contains functions for creating and working with lists.
    def sum(ints: NewList[Int]): Int = ints match { // A function that uses pattern matching to add up a list of integers
        case Nil => 0 // The sum of the empty list is 0.
        case Cons(x,xs) => x + sum(xs) // The sum of a list starting with `x` is `x` plus the sum of the rest of the list.
    }

    def product(ds: NewList[Double]): Double = ds match {
        case Nil => 1.0
        case Cons(0.0, _) => 0.0
        case Cons(x,xs) => x * product(xs)
    }

    def apply[A](as: A*): NewList[A] = // Variadic function syntax
        if (as.isEmpty) Nil
        else Cons(as.head, apply(as.tail: _*))

    def append[A](a1: NewList[A], a2: NewList[A]): NewList[A] =
        a1 match {
            case Nil => a2
            case Cons(h,t) => Cons(h, append(t, a2))
        }

    def foldRight[A,B](as: NewList[A], z: B)(f: (A, B) => B): B = // Utility functions
        as match {
            case Nil => z
            case Cons(x, xs) => f(x, foldRight(xs, z)(f))
        }

    def sum2(ns: NewList[Int]) =
        foldRight(ns, 0)((x,y) => x + y)

    def product2(ns: NewList[Double]) =
        foldRight(ns, 1.0)(_ * _) // `_ * _` is more concise notation for `(x,y) => x * y`; see sidebar

    // Exercise 3.2
    def tail[A](x: NewList[A]): NewList[A] = {
        x match {
            case Nil => sys.error("no tail exists in empty list")
            case Cons(_, t) => t
        }
    }
    
    // Exercise 3.3
    def setHead[A](l: NewList[A], h: A): NewList[A] = {
        l match {
            case Nil => sys.error("no first element to replace")
            case Cons(_, t) => Cons(h, t)
        }
    }

def drop[A](l: NewList[A], n: Int): NewList[A] = sys.error("todo")

def dropWhile[A](l: NewList[A], f: A => Boolean): NewList[A] = sys.error("todo")

def init[A](l: NewList[A]): NewList[A] = sys.error("todo")

def length[A](l: NewList[A]): Int = sys.error("todo")

def foldLeft[A,B](l: NewList[A], z: B)(f: (B, A) => B): B = sys.error("todo")

def map[A,B](l: NewList[A])(f: A => B): NewList[B] = sys.error("todo")
}


defined trait NewList
defined object Nil
defined class Cons
defined object NewList

#### Exercise 3.1

In [22]:
val x = NewList(1,2,3,4,5) match {
    case Cons(x, Cons(2, Cons(4, _))) => x
    case Nil => 42
    case Cons(x, Cons(y, Cons(3, Cons(4, _)))) => x + y
    case Cons(h, t) => h + NewList.sum(t)
    case _ => 101
}

x: Int = 3

This returns 3 because it hits the third case where we have (x,y,3,4,_) and returns x + y (1 + 2) here. The other cases don't match a list of 1:5

#### Exercise 3.2

In [24]:
val a = NewList(1,2,3)
NewList.tail(a)

a: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))
res23_1: NewList[Int] = Cons(2,Cons(3,Nil))

In [20]:
val a = NewList(1)
NewList.tail(a)

a: NewList[Int] = Cons(1,Nil)
res19_1: NewList[Int] = Nil

In [21]:
val a = NewList()
NewList.tail(a)

: 

#### Exercise 3.3

In [26]:
val a = NewList(1,2,3)
NewList.setHead(a, 10)

a: NewList[Int] = Cons(1,Cons(2,Cons(3,Nil)))
res25_1: NewList[Int] = Cons(10,Cons(2,Cons(3,Nil)))

In [27]:
val a = NewList(1)
NewList.setHead(a, 10)

a: NewList[Int] = Cons(1,Nil)
res26_1: NewList[Int] = Cons(10,Nil)

In [28]:
val a = NewList()
NewList.setHead(a, 10)

: 

#### Exercise 3.4

#### Exercise 3.5

#### Exercise 3.6

#### Exercise 3.7

#### Exercise 3.8

#### Exercise 3.9

#### Exercise 3.10

#### Exercise 3.11

#### Exercise 3.12